<a href="https://colab.research.google.com/github/HyunaShin/ImageStudy/blob/master/Plant_Seeding_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
api_token = {"username":"hyuna1615","key":"471ac110008706a0be1f1d4f780e4edd"}
! chmod 600 /root/.kaggle/kaggle.json
import json
import zipfile
import os

with open('/root/.kaggle/kaggle.json', 'w') as file:
   json.dump(api_token, file)
! kaggle competitions download -c plant-seedlings-classification

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
  0% 0.00/5.13k [00:00<?, ?B/s]
100% 5.13k/5.13k [00:00<00:00, 4.31MB/s]
 85% 73.0M/86.0M [00:01<00:00, 31.4MB/s]
100% 86.0M/86.0M [00:01<00:00, 50.9MB/s]
 99% 1.59G/1.60G [00:23<00:00, 61.9MB/s]
100% 1.60G/1.60G [00:23<00:00, 73.4MB/s]


In [27]:
! ls

adc.json       plant_seeding  sample_submission.csv.zip  train.zip
deep_learning  sample_data    test.zip


In [31]:
# ! unzip test.zip && unzip train.zip
! unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [32]:
import os
os.listdir()

['.config',
 'sample_submission.csv.zip',
 'adc.json',
 'sample_submission.csv',
 'train',
 'test',
 'train.zip',
 'test.zip',
 'deep_learning',
 'plant_seeding',
 'sample_data']

In [33]:
! cd train && ls

 Black-grass	    'Common wheat'	'Scentless Mayweed'
 Charlock	    'Fat Hen'		'Shepherds Purse'
 Cleavers	    'Loose Silky-bent'	'Small-flowered Cranesbill'
'Common Chickweed'   Maize		'Sugar beet'


In [0]:
#importing all the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import cv2
from glob import glob
import itertools


from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

scale = 70
seed = 7

In [41]:
#getting the data and resizing the images
path_to_images = "./train/*/*.png"
images = glob(path_to_images)

trainingset = []
traininglabels = []
num = len(images)
count = 1

for i in images:
  print(str(count)+'/'+str(num), end = "\r" )
  trainingset.append(cv2.resize(cv2.imread(i), (scale, scale)))
  traininglabels.append(i.split("/")[-2])
  count = count +1
 
trainingset = np.asarray(trainingset)
traininglabels = pd.DataFrame(traininglabels)

In [0]:
# trainingset